# Provena Dataset Registration with Metadata
This notebook will demonstrate how a Provena user can register a new dataset in the Provena Data Store.
The demonstration will include the prerequisites to register a dataset, the registration process it self, and post-registration activities such as updating metadata with new/missed or updated fields, releasing the dataset, creating new versions, changing access permissions, and more.

## Configuration Set up

In [2]:
# helpers for interacting with the datastore
import helpers.datastore as datastore
# This is a helper function for managing authentication with Provena
import mdsisclienttools.auth.TokenManager as ProvenaAuth
# Others
import json
from utils import pprint_json

In [3]:
# Provena config - replace with your Provena instance endpoints

# Replace the domain with the domain of your Provena instance
PROVENA_DOMAIN = "dev.rrap-is.com"

# Edit this to point to the Keycloak instance for your Provena instance
kc_endpoint = "https://auth.dev.rrap-is.com/auth/realms/rrap"

stage = "DEV"
registry_endpoint = "https://registry-api.{}".format(PROVENA_DOMAIN)
provenance_endpoint = "https://prov-api.{}".format(PROVENA_DOMAIN)
data_store_endpoint = "https://data-api.{}".format(PROVENA_DOMAIN)
job_endpoint =  "https://job-api.{}".format(PROVENA_DOMAIN)



In [4]:
# sets up auth connections - could potentially open browser window if not signed
# in recently - caches in .tokens.json - ensure this is included in gitignore
provena_auth = ProvenaAuth.DeviceFlowManager(
    stage=stage,
    keycloak_endpoint=kc_endpoint
)

# expose the get auth function which is used for provena methods 
get_auth = provena_auth.get_auth

No storage or object provided, using default location: .tokens.json.
Using storage type: FILE.
Using DEVICE auth flow.
Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



## Prerequisites to Dataset registration
Dataset metadata makes references to organisations, owners, and optionally more users (e.g. Data Custodian). These entities must be registered in the Provena Data Store before they can be referenced in a dataset registration. This is generally a one off activity and therefore is best performed using the the friendly web user interfaces. [A guide for registering entities is available](http://docs.provena.io/registry/registering_and_updating.html). Further more, you must also ensure you are registered as a Person Entity and your user account is linked to this enitity. More info below.

You are minimally required to register the following entities prior to dataset registration:
* **Person Entity** of yourself (for Provena to automatically assign your person entity as the dataset entity owner)
* (Optional) **Person Entity** of the Dataset's Data Custodian
* **Organisation Entity** of the Dataset's Record Creator Organisation
* **Organisation Entity** of the Dataset's Publisher


In addition to registering a Person Entity of yourself, you must also then [link your account to this Person Entity](http://docs.provena.io/getting-started-is/linking-identity.html).


### Pre-requisit entities

I have pre-registered the following entities in the web user-interface which generated the following references to be used in the dataset metadata fields later:  

| Entity Type and Purpose                       | Entity Handle + Link                                      |
|-----------------------------------|-------------------------------------------------|
| Person Entity of myself           | [10378.1/1764273](https://hdl.handle.net/10378.1/1764273)   |
| Person Entity of the Dataset's Data Custodian (Peter Baker)  | [10378.1/1758949](https://hdl.handle.net/10378.1/1758949)   |
| Organisation Entity of the Dataset's Record Creator Organisation (CSIRO)  | [10378.1/1764284](https://hdl.handle.net/10378.1/1764284)   |
| Organisation Entity of the Dataset's Publisher (CSIRO)  | [10378.1/1764284](https://hdl.handle.net/10378.1/1764284)   |



In [1]:
# TODO during demonstration.
record_creator = "10378.1/1764284" 
publisher = "10378.1/1764284"
data_custodian = "10378.1/1758949"

## Dataset Registration
Now the prerequisites are done. The following sections will demonstrate how to register a dataset in the Provena Data Store. You can check the expected json payload for the endpoints using https://data-api.dev.rrap-is.com/redoc. For registering a new dataset, see https://data-api.dev.rrap-is.com/redoc#tag/Register-dataset.

#### Load in Dataset Metadata

In [6]:
# Get path to file containing the dataset metadata

dataset_metadata_path = "configs/example_dataset_registration.json"

# load into dict
with open(dataset_metadata_path) as f:
    dataset_metadata = json.load(f)

# Inject references
dataset_metadata = datastore.inject_references(dataset_metadata, record_creator, publisher, data_custodian)

# Pretty Display
#pprint_json(dataset_metadata)

#### Post Dataset Metadata to Provena for Creation of Dataset Entity

In [7]:
register_response = datastore.register_dataset(datastore_endpoint=data_store_endpoint, dataset_metadata=dataset_metadata, auth=get_auth())
print(f"Registered dataset with id {register_response['handle']}")
pprint_json(register_response)

Token validation failed due to error: Signature has expired.
Refreshing using refresh token

Registering dataset with metadata: {'associations': {'organisation_id': '10378.1/1764284', 'data_custodian_id': '10378.1/1758949', 'point_of_contact': 'Lazaros'}, 'approvals': {'ethics_registration': {'relevant': False, 'obtained': False}, 'ethics_access': {'relevant': False, 'obtained': False}, 'indigenous_knowledge': {'relevant': False, 'obtained': False}, 'export_controls': {'relevant': False, 'obtained': False}}, 'dataset_info': {'name': 'The Test Dataset: A Mirror to the Soul of the Software', 'description': 'This is a test dataset purposed for demonstrating registration via API endpoint.', 'access_info': {'reposited': True}, 'publisher_id': '10378.1/1764284', 'created_date': '2022-10-02', 'published_date': '2023-10-03', 'license': 'https://creativecommons.org/licenses/by/4.0/', 'purpose': "But why, you might ask, was the Test Dataset so important? Well, dear reader, it served as a mirror 

In [8]:
print(register_response['handle'])

10378.1/1764755
